In [2]:
import numpy as np
import pandas as pd

In [ ]:
pip install catboost

In [41]:
from sklearn.model_selection import StratifiedKFold     #k-fold cross-validation
from sklearn.feature_extraction.text import TfidfVectorizer #feature extraction module; converts raw documents (text) into a matrix of TF-IDF features
from sklearn.decomposition import TruncatedSVD              #decomposition module; used for dimensionality reduction of sparse data
from catboost import CatBoostClassifier, Pool               #gradient boosting library by Yandex that handles categorical features naturally. Pool is used to hold data and can be passed as an argument for training a CatBoost model
from catboost.utils import eval_metric                  #evaluation metrics for CatBoost models, such as accuracy, AUC (Area Under the Curve)
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
import warnings
warnings.filterwarnings("ignore")

In [66]:
RAND_VAL=32
num_folds=20 ## Number of folds
n_est=500 ## Number of estimators

In [ ]:
df_train = pd.read_csv('train.csv')
df_train.head()

In [9]:
df_test = pd.read_csv('test.csv')
df_test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [19]:
for feat in df_train.columns:
  df_train[feat] = df_train[feat].fillna('')
for feat in df_test.columns:
  df_test[feat] = df_test[feat].fillna('')
feat_cols=df_train.columns.drop(['target'])
X=df_train[feat_cols]
y=df_train['target']
cat_features = np.where(X.dtypes != np.float64)[0]
print(cat_features) #everything is categorical

[0 1 2 3]


In [67]:
folds = StratifiedKFold(n_splits=num_folds, random_state=RAND_VAL, shuffle=True)

best_f1_scores = []
optimal_thresholds = []

for n_fold, (train_idx, valid_idx) in enumerate(folds.split(X, y)):
    X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
    X_val, y_val = X.iloc[valid_idx], y.iloc[valid_idx]

    train_pool = Pool(X_train, y_train, cat_features=cat_features)
    val_pool = Pool(X_val, y_val, cat_features=cat_features)

    clf = CatBoostClassifier(
        eval_metric='BalancedAccuracy',  # or 'Logloss', 'Accuracy'
        custom_metric='AOC',
        task_type='GPU',
        learning_rate=0.02,
        iterations=n_est
    )

    clf.fit(train_pool, eval_set=val_pool, verbose=300)

    # Predict probabilities
    y_pred_val_prob = clf.predict_proba(X_val)[:, 1]

    # Optimize the threshold for the best F1 score
    #thresholds = np.linspace(0, 1, 101)
    #f1_scores = [f1_score(y_val, y_pred_val_prob > thresh) for thresh in thresholds]
    #best_thresh = thresholds[np.argmax(f1_scores)]
    #best_f1 = max(f1_scores)
    #best_f1_scores.append(best_f1)
    #optimal_thresholds.append(best_thresh)

    print(f"Best F1 Score for fold {n_fold}: {best_f1} at threshold {best_thresh}")

    print("----------------")


Default metric period is 5 because BalancedAccuracy is/are not implemented for GPU
Metric BalancedAccuracy is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	learn: 0.6990337	test: 0.7400809	best: 0.7400809 (0)	total: 78.2ms	remaining: 39s
300:	learn: 0.7020851	test: 0.7446190	best: 0.7469231 (240)	total: 9.61s	remaining: 6.35s
499:	learn: 0.7036588	test: 0.7461785	best: 0.7469231 (240)	total: 15s	remaining: 0us
bestTest = 0.7469231202
bestIteration = 240
Shrink model to first 241 iterations.
Best F1 Score for fold 0: 0.7549295774647887 at threshold 0.37
----------------


Default metric period is 5 because BalancedAccuracy is/are not implemented for GPU
Metric BalancedAccuracy is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	learn: 0.6964862	test: 0.7630522	best: 0.7630522 (0)	total: 34.4ms	remaining: 17.2s
300:	learn: 0.6999033	test: 0.7661009	best: 0.7661009 (5)	total: 10.6s	remaining: 7.03s
499:	learn: 0.7019953	test: 0.7745026	best: 0.7745026 (480)	total: 16.5s	remaining: 0us
bestTest = 0.7745026413
bestIteration = 480
Shrink model to first 481 iterations.
Best F1 Score for fold 1: 0.7572254335260116 at threshold 0.42
----------------


Default metric period is 5 because BalancedAccuracy is/are not implemented for GPU
Metric BalancedAccuracy is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


...

0:	learn: 0.7022757	test: 0.7114585	best: 0.7114585 (0)	total: 32.4ms	remaining: 16.2s
300:	learn: 0.7057355	test: 0.7007577	best: 0.7114585 (0)	total: 8.54s	remaining: 5.64s
499:	learn: 0.7064957	test: 0.7068927	best: 0.7114585 (0)	total: 15.6s	remaining: 0us
bestTest = 0.7114585395
bestIteration = 0
Shrink model to first 1 iterations.
Best F1 Score for fold 17: 0.6727272727272727 at threshold 0.5
----------------


Default metric period is 5 because BalancedAccuracy is/are not implemented for GPU
Metric BalancedAccuracy is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	learn: 0.6917752	test: 0.7061576	best: 0.7061576 (0)	total: 34.8ms	remaining: 17.4s
300:	learn: 0.7006471	test: 0.7061293	best: 0.7207034 (10)	total: 10.4s	remaining: 6.85s
499:	learn: 0.7028223	test: 0.7053660	best: 0.7207034 (10)	total: 15.5s	remaining: 0us
bestTest = 0.7207034011
bestIteration = 10
Shrink model to first 11 iterations.
Best F1 Score for fold 18: 0.6940874035989717 at threshold 0.46
----------------


Default metric period is 5 because BalancedAccuracy is/are not implemented for GPU
Metric BalancedAccuracy is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	learn: 0.7003384	test: 0.6800204	best: 0.6800204 (0)	total: 31.6ms	remaining: 15.8s
300:	learn: 0.7038954	test: 0.6861695	best: 0.6907636 (245)	total: 10.3s	remaining: 6.81s
499:	learn: 0.7062270	test: 0.6861695	best: 0.6907636 (245)	total: 15.8s	remaining: 0us
bestTest = 0.6907636199
bestIteration = 245
Shrink model to first 246 iterations.
Best F1 Score for fold 19: 0.6931818181818182 at threshold 0.4
----------------


In [68]:
#average_f1_score = np.mean(best_f1_scores)
#average_best_thresh = np.mean(optimal_thresholds)
#print(f"Average F1 Score across folds: {average_f1_score}")
#print(f"Average Optimal Threshold across folds: {average_best_thresh}")

# Predict on test data using the average optimal threshold
y_pred_test_prob = clf.predict_proba(df_test[feat_cols])[:, 1]
y_pred_test = (y_pred_test_prob > 0.5).astype(int)



In [69]:
prediction = pd.DataFrame({
    'id': df_test['id'],
    'target': y_pred_test
})

prediction.to_csv("predictions",index = False)
prediction.head()

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1
